In [1]:
!pip install evadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# !wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
import os
import time

def shell(command):
    print(command)
    os.system(command)

def stop_eva_server():
    shell('[  -z "$(lsof -ti:5432)" ] || kill -9 $(lsof -ti:5432)"')
  
def launch_eva_server():
  # Stop EVA server if it is running
    stop_eva_server()

    os.environ['GPU_DEVICES'] = '0'

    # Start EVA server
    shell("nohup eva_server > eva.log 2>&1 &")

    try:
        with open('eva.log', 'r') as f:
            print(f.read())
    except FileNotFoundError:
        pass

    # Wait for server to start
    time.sleep(10)

def connect_to_server():
    from eva.server.db_api import connect
    %pip install nest_asyncio --quiet
    import nest_asyncio
    nest_asyncio.apply()

    # Connect client with server
    connection = connect(host = '127.0.0.1', port = 5432) 
    cursor = connection.cursor()

    return cursor
  
# Launch server
launch_eva_server()
cursor = connect_to_server()
    
  

[  -z "$(lsof -ti:5432)" ] || kill -9 $(lsof -ti:5432)"
nohup eva_server > eva.log 2>&1 &
12-19-2022 16:13:22 INFO  [catalog_manager:catalog_manager.py:_bootstrap_catalog:0072] Bootstrapping catalog
12-19-2022 16:13:22 INFO  [base_model:base_model.py:init_db:0103] Creating tables
12-19-2022 16:13:22 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0895 sec
12-19-2022 16:13:22 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF FastRCNNObjectDetector already exists, nothing added.
12-19-2022 16:13:22 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0258 sec
12-19-2022 16:13:22 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Array_Count already exists, nothing added.
12-19-2022 16:13:22 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0184 sec
12-19-2022 16:13:22 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Crop already exists, nothing added.
12-19-2022 16:13:22 INFO  [timer:timer.p

In [1]:
# Getting MNIST as a video
!wget -nc https://www.dropbox.com/s/yxljxz6zxoqu54v/mnist.mp4
# Getting a udf
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/apps/mnist/eva_mnist_udf.py

--2022-12-19 16:26:06--  https://www.dropbox.com/s/yxljxz6zxoqu54v/mnist.mp4
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/yxljxz6zxoqu54v/mnist.mp4 [following]
--2022-12-19 16:26:06--  https://www.dropbox.com/s/raw/yxljxz6zxoqu54v/mnist.mp4
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e71830a05a3100370ec515fe9.dl.dropboxusercontent.com/cd/0/inline/By514ui1umDHbnuvoFqQszqvERJTjKX8ZF25f56bjnGTG0Qd-OFZtmXXHwgS9_sao_ZIJUHywio7_ktG5R4B2eEntZ93jCj1bvqPFPsrApfJP1xF60TU0mumAya9NIEoMAQVGivCbenZQA9brfVlndENW5Fvt9ISLH5sFss66kvOQA/file# [following]
--2022-12-19 16:26:06--  https://uc7e71830a05a3100370ec515fe9.dl.dropboxusercontent.com/cd/0/inline/By514ui1umDHbnuvoFqQszqvERJTjKX8ZF25f56bjnGTG0Qd-OFZtmXXHwgS9_sao_ZIJUHywio7_ktG5R4B2eEnt

In [13]:
import numpy as np
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
from glob import glob

In [6]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 2.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pydicom

In [9]:
def load_scan(paths):
  slices = [pydicom.read_file(path) for path in paths]
  slices.sort(key = lambda x: int(x.InstanceNumber), reverse = True)
  try:
    slice_thickness = np.abs(slices[0].ImagePositionSample[2] - slices[1].ImagePositionSample[2])
  except:
    slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
  
  for s in slices:
    s.SliceThickness = slice_thickness

  return slices

  def get_piexels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    image[image == -2000] = 0

    image += np.int16(intercept)

    return np.array(image, dtype = np.int16)



In [10]:
pip install chart_studio

     |████████████████████████████████| 64 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 15.3 MB 877 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [11]:
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops

from scipy.linalg import norm
import scipy.ndimage

from ipywidgets.widgets import *
import ipywidgets as widgets

import plotly
from plotly.graph_objs import *
import chart_studio

import asyncio
import sys

In [12]:
import asyncio
import sys

from typing import List
from eva.server.db_api import connect, connect_async

async def run_async(query: List[str]):
    hostname = '0.0.0.0'
    port = 5432

    connection = await connect_async(hostname, port)
    cursor = connection.cursor()
    for onequery in query:
        await cursor.execute_async(onequery)
        response = await cursor.fetch_one_async()
        print('Query: %s' % onequery)
        print(response)

def run(query: List[str]):
    hostname = '0.0.0.0'
    port = 5432

    connection = connect(hostname, port)
    cursor = connection.cursor()
    for onequery in query:
        cursor.execute(onequery)
        response = cursor.fetch_one()
        print('Query: %s' % onequery)
        print(response)

if __name__ == '__main__':
    queries = ['INVALID QUERY',
               'UPLOAD INFILE "data/ua_detrac/ua_detrac.mp4" \
                       PATH "test_video.mp4";',
               'LOAD DATA INFILE "test_video.mp4" INTO MyVideo;',
               'SELECT id,data FROM MyVideo WHERE id < 5;']

    if sys.argv[1] != 'sync':
        asyncio.run(run_async(queries))
    else:
        run([queries])

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
sample_id = '00003'
sample_folder = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{sample_id}/'
data_paths = glob(sample_folder + '/*/*.dcm')
print (f'Total of {len(data_paths)} DICOM images.\nFirst 5 filenames:' )
data_paths[:5]